# This is Jeopardy!

#### Overview

This project is slightly different than others you have encountered thus far. Instead of a step-by-step tutorial, this project contains a series of open-ended requirements which describe the project you'll be building. There are many possible ways to correctly fulfill all of these requirements, and you should expect to use the internet, Codecademy, and/or other resources when you encounter a problem that you cannot easily solve.

#### Project Goals

You will work to write several functions that investigate a dataset of _Jeopardy!_ questions and answers. Filter the dataset for topics that you're interested in, compute the average difficulty of those questions, and train to become the next Jeopardy champion!

## Prerequisites

In order to complete this project, you should have completed the Pandas lessons in the <a href="https://www.codecademy.com/learn/paths/analyze-data-with-python">Analyze Data with Python Skill Path</a>. You can also find those lessons in the <a href="https://www.codecademy.com/learn/data-processing-pandas">Data Analysis with Pandas course</a> or the <a href="https://www.codecademy.com/learn/paths/data-science/">Data Scientist Career Path</a>.

Finally, the <a href="https://www.codecademy.com/learn/practical-data-cleaning">Practical Data Cleaning</a> course may also be helpful.

## Project Requirements

1. We've provided a csv file containing data about the game show _Jeopardy!_ in a file named `jeopardy.csv`. Load the data into a DataFrame and investigate its contents. Try to print out specific columns.

   Note that in order to make this project as "real-world" as possible, we haven't modified the data at all - we're giving it to you exactly how we found it. As a result, this data isn't as "clean" as the datasets you normally find on Codecademy. More specifically, there's something odd about the column names. After you figure out the problem with the column names, you may want to rename them to make your life easier for the rest of the project.
   
   In order to display the full contents of a column, we've added this line of code for you:
   
   ```py
   pd.set_option('display.max_colwidth', None)
   ```

In [10]:
import pandas as pd
import numpy as np
import re
pd.set_option('display.max_colwidth', None)

In [178]:
# Import the data into a dataframe
jeopardy = pd.read_csv('jeopardy.csv', skipinitialspace=True) #delete any leading spaces in columns
#print(jeopardy.head(5))

In [ ]:
#Let's take a look at the column names
columns = list(jeopardy.columns)
print(columns)

#convert middle spaces in column names to '_'
for index in range(0,len(columns)):
    columns[index] = columns[index].replace(' ','_')
    print(columns[index])
jeopardy.columns = columns
print(jeopardy.columns)
print(jeopardy.dtypes)
print(jeopardy.head(5))

In [189]:
# Convert the Air_Date data to datetime format
jeopardy['Air_Date'] = pd.to_datetime(jeopardy['Air_Date'], yearfirst=True, format="%Y-%m-%d")
jeopardy['Year'] = pd.DatetimeIndex(jeopardy.Air_Date).year

# Convert the Value to an int64 datatype
for index in range(0,len(jeopardy.Value)):
    if jeopardy.Value.iat[index] =='None':
        jeopardy.Value.iat[index] = 0.0
jeopardy['Value'] = jeopardy['Value'].replace('[\$\,\.]','',regex=True)
jeopardy['Value'] = pd.to_numeric(jeopardy['Value'])


#print(jeopardy.head(5))
print(jeopardy.dtypes)

Show_Number             int64
Air_Date       datetime64[ns]
Round                  object
Category               object
Value                 float64
Question               object
Answer                 object
Year                    int64
dtype: object


In [184]:
print(jeopardy.head(5))

   Show_Number   Air_Date      Round                         Category  Value  \
0         4680 2004-12-31  Jeopardy!                          HISTORY  200.0   
1         4680 2004-12-31  Jeopardy!  ESPN's TOP 10 ALL-TIME ATHLETES  200.0   
2         4680 2004-12-31  Jeopardy!      EVERYBODY TALKS ABOUT IT...  200.0   
3         4680 2004-12-31  Jeopardy!                 THE COMPANY LINE  200.0   
4         4680 2004-12-31  Jeopardy!              EPITAPHS & TRIBUTES  200.0   

                                                                                                      Question  \
0             For the last 8 years of his life, Galileo was under house arrest for espousing this man's theory   
1  No. 2: 1912 Olympian; football star at Carlisle Indian School; 6 MLB seasons with the Reds, Giants & Braves   
2                     The city of Yuma in this state has a record average of 4,055 hours of sunshine each year   
3                         In 1963, live on "The Art Linkletter 

2. Write a function that filters the dataset for questions that contains all of the words in a list of words. For example, when the list `["King", "England"]` was passed to our function, the function returned a DataFrame of 49 rows. Every row had the strings `"King"` and `"England"` somewhere in its `" Question"`.

   Test your function by printing out the column containing the question of each row of the dataset.

### Research Notes
You need to use lookahead as some of the other responders have said, but the lookahead has to account for other characters between its target word and the current match position. For example:

(?=.*word1)(?=.*word2)(?=.*word3)

The .* in the first lookahead lets it match however many characters it needs to before it gets to "word1". Then the match position is reset and the second lookahead seeks out "word2". Reset again, and the final part matches "word3"; since it's the last word you're checking for, it isn't necessary that it be in a lookahead, but it doesn't hurt.

In order to match a whole paragraph, you need to anchor the regex at both ends and add a final .* to consume the remaining characters. Using Perl-style notation, that would be:

/^(?=.*word1)(?=.*word2)(?=.*word3).*$/m

The 'm' modifier is for multline mode; it lets the ^ and $ match at paragraph boundaries ("line boundaries" in regex-speak). It's essential in this case that you not use the 's' modifier, which lets the dot metacharacter match newlines as well as all other characters.

Finally, you want to make sure you're matching whole words and not just fragments of longer words, so you need to add word boundaries:

/^(?=.*\bword1\b)(?=.*\bword2\b)(?=.*\bword3\b).*$/m

In [193]:
#create a function to turn a list of words to search for into a pattern, assuming ALL words in the list should be found.
def list_to_pattern(some_list):
    pattern =''
    searchterm = "(?=.*(\\b{word}\\b|\\b{word2}\\b))"
    for item in some_list:
        new_term = searchterm.format(word = item, word2 = item.lower())
        pattern+=new_term
        
    return pattern



def find_words(data, match_list):
    pattern = list_to_pattern(match_list) #calls the function that converts the user's input list to a search pattern
    print('This program will use ' + pattern + ' to search the dataset.')
    # Now that we have the regex pattern to search for, let's get crackin!
    
    # initialize a dataframe to put the resulting data from the search
    found_list = pd.DataFrame(columns = ['Show_Number', 'Air_Date', 'Round', 'Category', 'Value', 'Question',
       'Answer','Year'])
   
    for index in range(0,len(jeopardy.Question)):
        if re.search(r'{}'.format(pattern),jeopardy.Question.iloc[index]): # if the search finds the pattern in the Question...
            s = jeopardy.iloc[index,:].to_frame().transpose() #take that row and transpose it
            found_list = pd.concat([found_list,s]) # add it to the newly created dataframe above
        
    return found_list

3. Test your original function with a few different sets of words to try to find some ways your function breaks. Edit your function so it is more robust.

   For example, think about capitalization. We probably want to find questions that contain the word `"King"` or `"king"`.
   
   You may also want to check to make sure you don't find rows that contain substrings of your given words. For example, our function found a question that didn't contain the word `"king"`, however it did contain the word `"viking"` &mdash; it found the `"king"` inside `"viking"`. Note that this also comes with some drawbacks &mdash; you would no longer find questions that contained words like `"England's"`.

In [177]:
found_list = find_words(jeopardy,['Computer','Mouse'])
print(len(found_list))

This program will use (?=.*\b(Computer\b|\bcomputer\b))(?=.*\b(Mouse\b|\bmouse\b)) to search the dataset.
7


4. We may want to eventually compute aggregate statistics, like `.mean()` on the `" Value"` column. But right now, the values in that column are strings. Convert the`" Value"` column to floats. If you'd like to, you can create a new column with float values.

   Now that you can filter the dataset of question, use your new column that contains the float values of each question to find the "difficulty" of certain topics. For example, what is the average value of questions that contain the word `"King"`?
   
   Make sure to use the dataset that contains the float values as the dataset you use in your filtering function.

In [172]:
king_list = find_words(jeopardy,['King']).reset_index(drop=True)
king_count = king_list.Answer.value_counts()
print(find_average(king_count, king_list))
print('The average value of a question with the word \'King\' is $'+ str(king_list.Value.mean()))
print('\nThe max value of a question containing the word \'King\' is $' + str(king_list.Value.max()))
print('\nThe min value of a question containing the word \'King\' is $' + str(king_list.Value.min()))

This program will use (?=.*\b(King\b|\bking\b)) to search the dataset.
The average value of a question with the word 'King' is $817.7616926503341

The max value of a question containing the word 'King' is $9000.0

The min value of a question containing the word 'King' is $100.0


5. Write a function that returns the count of unique answers to all of the questions in a dataset. For example, after filtering the entire dataset to only questions containing the word `"King"`, we could then find all of the unique answers to those questions. The answer "Henry VIII" appeared 55 times and was the most common answer.

#### The following finds the average Value for subset of the main dataset with a list of answers

In [176]:
# Final Jeopardy questions do not have an associated Value. To find the average, let's drop these rows from the dataset. 
# Note, this will modify the existing dataset. May want to establish a new one for finding averages.
def drop_final(data):
    final_rows = data[data.Round == 'Final Jeopardy!'].index
    data.drop(final_rows, inplace=True)
    
    return data

def find_average(items, data):
    data = drop_final(data).reset_index(drop=True)
    item_df = pd.DataFrame(items, columns = ['Answer'])
    item_df['Total'] = 0.0
    item_df['Average'] = 0.0
    item_df['Count'] = 0.0 
    items = list(item_df.index.values)
    
    for data_index in range(0,len(data.Answer)):   
        for item in items:
            if item == data.loc[data_index,'Answer']:
                item_df.loc[item,'Count'] = item_df.loc[item,'Count'] + 1
                item_df.loc[item,'Total'] = item_df.loc[item,'Total'] + data.loc[data_index,'Value']
                    
    item_df['Average'] = item_df['Total']/item_df['Count']
    item_df['Final_Jeopardy_Count'] = item_df['Answer'] - item_df['Count']
    return item_df

6. Explore from here! This is an incredibly rich dataset, and there are so many interesting things to discover. There are a few columns that we haven't even started looking at yet. Here are some ideas on ways to continue working with this data:

 * Investigate the ways in which questions change over time by filtering by the date. How many questions from the 90s use the word `"Computer"` compared to questions from the 2000s?
 * Is there a connection between the round and the category? Are you more likely to find certain categories, like `"Literature"` in Single Jeopardy or Double Jeopardy?
 * Build a system to quiz yourself. Grab random questions, and use the <a href="https://docs.python.org/3/library/functions.html#input">input</a> function to get a response from the user. Check to see if that response was right or wrong.

##### Setting up some searches to breakdown the data by decade.

Looking for data on the word 'Computer' in the question

In [215]:
word = ['Computer']
data_search = find_words(jeopardy, word)
data_search = data_search.reset_index(drop=True)
data_search['Decade'] = data_search.apply(lambda x:('1990s' if x.Year < 2000 else '2000s'), axis=1)
counts = data_search.Decade.value_counts()
print(counts)

This program will use (?=.*(\bComputer\b|\bcomputer\b)) to search the dataset.
2000s    278
1990s     84
Name: Decade, dtype: int64


Looking for data on the word 'Book' in the question

In [212]:
word = ['Book']
data_search = find_words(jeopardy, word)
data_search = data_search.reset_index(drop=True)
data_search['Decade'] = data_search.apply(lambda x:('1990s' if x.Year < 2000 else '2000s'), axis=1)
counts = data_search.Decade.value_counts()
print(counts)

2000s    1752
1990s     788
Name: Decade, dtype: int64


Looking for data on the word 'Cellphone'

In [231]:
word = ['Bible']
data_search = find_words(jeopardy, word)
data_search = data_search.reset_index(drop=True)
data_search['Decade'] = data_search.apply(lambda x:('1990s' if x.Year < 2000 else '2000s'), axis=1)
counts = data_search.Decade.value_counts()
print(counts)

This program will use (?=.*(\bBible\b|\bbible\b)) to search the dataset.
2000s    225
1990s     93
Name: Decade, dtype: int64


##### Taking a look at data by category

In [232]:
df = jeopardy.loc[jeopardy['Category'] == 'BIBLE'].groupby(['Round']).count()
print(df)

Empty DataFrame
Columns: [Show_Number, Air_Date, Category, Value, Question, Answer, Year]
Index: []


## Solution

7. Compare your program to our <a href="https://content.codecademy.com/PRO/independent-practice-projects/jeopardy/jeopardy_solution.zip">sample solution code</a> - remember, that your program might look different from ours (and probably will) and that's okay!

8. Great work! Visit <a href="https://discuss.codecademy.com/t/this-is-jeopardy-challenge-project-python-pandas/462365">our forums</a> to compare your project to our sample solution code. You can also learn how to host your own solution on GitHub so you can share it with other learners! Your solution might look different from ours, and that's okay! There are multiple ways to solve these projects, and you'll learn more by seeing others' code.